FUNCTION
===================

TIME AND DATE
===================

INPUT(): "reread" a variable according to a new informat, char to numeric (IN for INteger)  
take the first argument and "reads" it as if it were coming from a file, according to the INFORMAT listed as the second argument.  

PUT: use formatted value of a variable to create new variable, numeric to char 
it "writes out" the value of the first argument, according to the FORMAT in second argument. By "write out" we actually mean assign the value to the variable on the left of the equal sign.  

STRING
=========
concantenate: `||`

COMPBL(var) # compress multi-blank into single blank
COMPRESS(VAR,'charToRm')
SUBSTR(char_var,start,length)
INDEX('ABCDEFG','DEF'); #return 4
LENGTH() #trailing blacks are excluded in computation;
VERIFY(ANSWER, 'ABCDE'); # make sure answer has only ABCDE
SCAN(char_var,n,'list-of-delimiters'); #SCAN("a-b,c;d" ,2,'-,;'); RETURN b

In [1]:

DATA EX_10; 
   INPUT LONG_STR $ 1-80;
   ARRAY PIECES[5] $ 10   
      PIECE1-PIECE5;
   DO I = 1 TO 5;
      PIECES[I] = SCAN(LONG_STR,I,',;.! ');
   END;
   DROP LONG_STR I;
DATALINES4;
THIS LINE,CONTAINS!FIVE.WORDS
ABCDEFGHIJKL XXX;YYY
;;;;
PROC PRINT DATA=EX_10 NOOBS;
   TITLE 'Listing of Example 10';
RUN;

PIECE1,PIECE2,PIECE3,PIECE4,PIECE5
THIS,LINE,CONTAINS,FIVE,WORDS
ABCDEFGHIJ,XXX,YYY,,


In [2]:

DATA EX_12;
   LENGTH A B C D E $ 1;
   INPUT A B C D E X Y;
DATALINES;
M f P p D 1 2
m f m F M 3 4
;
DATA UPPER;
   SET EX_12;
   ARRAY ALL_C[*] _CHARACTER_;
   DO I = 1 TO DIM(ALL_C);
      ALL_C[I] = UPCASE(ALL_C[I]);
   END;
   DROP I;
RUN;

PROC PRINT DATA=UPPER NOOBS;
   TITLE 'Listing of UPPER';
RUN;

A,B,C,D,E,X,Y
M,F,P,P,D,1,2
M,F,M,F,M,3,4


In [6]:

DATA EX_13;
   INPUT QUES : $1. @@;
   QUES = TRANSLATE(QUES,'ABCDE','12345');
DATALINES;
1 4 3 2 5
5 3 4 2 1
;
PROC PRINT DATA=EX_13(obs=3) NOOBS;
   TITLE 'LISTING OF EXAMPLE 13';
RUN;


QUES
A
D
C


In [7]:
DATA CONVERT;
   INPUT @1 ADDRESS $20. ;
   *** Convert Street, Avenue, and Boulevard to their abbreviations;
   ADDRESS = TRANWRD (ADDRESS,'Street','St.');
   ADDRESS = TRANWRD (ADDRESS,'Avenue','Ave.');
   ADDRESS = TRANWRD (ADDRESS,'Road','Rd.');
DATALINES;
89 Lazy Brook Road 
123 River Rd.
12 Main Street
;
PROC PRINT DATA=CONVERT;
   TITLE 'Listing of Data Set CONVERT';
RUN;

Obs,ADDRESS
1,89 Lazy Brook Rd.
2,123 River Rd.
3,12 Main St.


LAG & DIFF
=============
LAGn()  
DIFn()

TEMPLATE
============

Values as % of Grand Mean
--------------------------------
Use '_N_' to conditionally exec. SET stmt, any variable coming in from a SET stmt are automatically retained so the values of MHR, MSBP, MDBP will not change and will be avilable for every iteration of DATA STEP.

In [9]:
DATA TEST;
   INPUT HR SBP DBP;
DATALINES;
80 160 100
70 150 90
60 140 80
;
PROC MEANS NOPRINT DATA=TEST;
   VAR HR SBP DBP;
   OUTPUT OUT=MOUT(DROP=_TYPE_ _FREQ_)
          MEAN=MHR MSBP MDBP;
RUN;

DATA NEW (DROP=MHR MSBP MDBP);
   SET TEST;
   IF _N_ = 1 THEN SET MOUT;
   HRPER=100*HR/MHR;
   SBPPER=100*SBP/MSBP;
   DBPPER=100*DBP/MDBP;
RUN;

PROC PRINT DATA=NEW NOOBS;
   TITLE 'Listing of Data Set NEW';
RUN;      

HR,SBP,DBP,HRPER,SBPPER,DBPPER
80,160,100,114.286,106.667,111.111
70,150,90,100.000,100.000,100.000
60,140,80,85.714,93.333,88.889


Value as % of Group Mean
------------------------------------

In [14]:

DATA TEST; 
   INPUT GROUP $ HR SBP DBP @@; 
DATALINES; 
A 80 160 100 A 70 150 90 A 60 140 80 
B 90 200 180 B 80 180 140 B 70 140 80 
; 
PROC SORT DATA=TEST; 
   BY GROUP;
RUN;

PROC MEANS DATA=TEST NOPRINT NWAY;
   CLASS GROUP;
   VAR HR SBP DBP;
   OUTPUT OUT=MOUT(DROP=_TYPE_ _FREQ_)
          MEAN=MHR MSBP MDBP;
RUN;

DATA NEW (DROP=MHR MSBP MDBP);
   MERGE TEST MOUT;
      BY GROUP;
   HRPER=100*HR/MHR;
   SBPPER=100*SBP/MSBP;
   DBPPER=100*DBP/MDBP;
RUN;

PROC PRINT NOOBS;
   TITLE 'Listing of Data Set NEW';
RUN;                                          


GROUP,HR,SBP,DBP,HRPER,SBPPER,DBPPER
A,80,160,100,114.286,106.667,111.111
A,70,150,90,100.000,100.000,100.000
A,60,140,80,85.714,93.333,88.889
B,90,200,180,112.500,115.385,135.000
B,80,180,140,100.000,103.846,105.000
B,70,140,80,87.500,80.769,60.000


Plotting Means with Error Bars
---------------------------------

Macro
------------

In [15]:

DATA TEST;
   %LET LIST=ONE TWO;
   INPUT &LIST THREE FOUR;
DATALINES;
1 2 3 4
4 5 6 6
;
PROC FREQ DATA=TEST;
   TABLES &LIST;
RUN;                    

ONE,Frequency,Percent,CumulativeFrequency,CumulativePercent
1,1,50.00,1,50.00
4,1,50.00,2,100.00
TWO,Frequency,Percent,CumulativeFrequency,CumulativePercent
2,1,50.00,1,50.00
5,1,50.00,2,100.00


Relative Frequencies
--------------------------

In [17]:
DATA ICD;
   INPUT ID YEAR ICD;
DATALINES;
001 1950 450
002 1950 440
003 1951 460
004 1950 450
005 1951 300
;
PROC FREQ DATA=ICD;
   TABLES YEAR*ICD / OUT=ICDFREQ NOPRINT;
   ***Data set ICDFREQ contains the counts 
   for each CODE in each YEAR;
RUN;

PROC FREQ DATA=ICD;
   TABLES YEAR / OUT=TOTAL NOPRINT;
   ***Data set ICD contains the total number 
   of obs for each YEAR;
RUN;

DATA RELATIVE;
   MERGE ICDFREQ TOTAL (RENAME=(COUNT=TOT_CNT));
   ***We need to rename COUNT in one of the two data sets
      so that we can have both values in data set RELATIVE;
   BY YEAR;
   RELATIVE=100*COUNT/TOT_CNT;
   DROP PERCENT;
RUN;

PROC PRINT DATA=RELATIVE;
   TITLE 'Relative Frequencies of ICD Codes by Year';
RUN; 

Obs,YEAR,ICD,COUNT,TOT_CNT,RELATIVE
1,1950,440,1,3,33.3333
2,1950,450,2,3,66.6667
3,1951,300,1,2,50.0000
4,1951,460,1,2,50.0000


Combined Freq
-------------

In [18]:

PROC FORMAT;
   VALUE SYMPTOM 1='ALCOHOL' 2='INK' 3='SULPHUR' 4='IRON' 5='TIN'
                 6='COPPER'  7='DDT' 8='CARBON' 9='SO2' 10='NO2';
RUN;

DATA SENSI;
   INPUT ID 1-4 (CHEM1-CHEM10)(1.);
   ARRAY CHEM[*] CHEM1-CHEM10;
   DO I=1 TO 10;
      IF CHEM[I]=1 THEN DO;
         SYMPTOM=I;
         OUTPUT;
      END;
   END;
   KEEP ID SYMPTOM;
   FORMAT SYMPTOM SYMPTOM.;
DATALINES;
00011010101010
00021000010000
00031100000000
00041001001111
00051000010010
;
PROC FREQ DATA=SENSI ORDER=FREQ;
   TABLES SYMPTOM;
RUN;                            


SYMPTOM,Frequency,Percent,CumulativeFrequency,CumulativePercent
ALCOHOL,5,27.78,5,27.78
SO2,3,16.67,8,44.44
COPPER,2,11.11,10,55.56
DDT,2,11.11,12,66.67
CARBON,1,5.56,13,72.22
INK,1,5.56,14,77.78
IRON,1,5.56,15,83.33
NO2,1,5.56,16,88.89
SULPHUR,1,5.56,17,94.44
TIN,1,5.56,18,100.00


Moving Average
----------------

In [20]:
DATA MOVING;
   INPUT COST @@;
   DAY+1;
   COST1=LAG(COST);
   COST2=LAG2(COST);
   IF _N_ GE 3 THEN MOV_AVE = MEAN (OF COST COST1 COST2);
   DROP COST1 COST2;
DATALINES;
1 2 3 4 . 6 8 12 8
;
PROC PRINT DATA=MOVING NOOBS;
   TITLE 'Listing of Data Set MOVING';
RUN;                                   

COST,DAY,MOV_AVE
1,1,.
2,2,.
3,3,2.00000
4,4,3.00000
.,5,3.50000
6,6,5.00000
8,7,7.00000
12,8,8.66667
8,9,9.33333


 Sort within an Observation 
 --------------------------

In [23]:

DATA TEST;
   INPUT ID L1-L5;
DATALINES;
1 1 2 3 4 5
2 1 3 5 2 4
3 . 7 9 . .
;
DATA NEW;
   SET TEST;
   ARRAY L[5] L1-L5;
   LOOP: FLAG=0;   
   DO I=1 TO 4;  
      IF L[I+1] GT L[I] THEN DO;  
         HOLD=L[I];
         L[I]=L[I+1];
         L[I+1]=HOLD;
         FLAG=1;
      END;
   END;
   IF FLAG=1 THEN GO TO LOOP;
   DROP I FLAG HOLD;
RUN;

proc print data=NEW(OBS=3);
RUN;

Obs,ID,L1,L2,L3,L4,L5
1,1,5,4,3,2,1
2,2,5,4,3,2,1
3,3,9,7,.,.,.


In [24]:

DATA NEW;
   SET TEST;
   ARRAY L[5] L1-L5;
   FLAG = 1;
   DO UNTIL (FLAG = 0);
      FLAG = 0;
      DO I = 1 TO 4;
         IF L[I+1] GT L[I] THEN DO;
            HOLD = L[I];
            L[I] = L[I+1];
            L[I+1] = HOLD;
            FLAG = 1;
         END;
      END;
   END;
   DROP I FLAG HOLD;
RUN;

proc print data=NEW(OBS=3);
RUN;

Obs,ID,L1,L2,L3,L4,L5
1,1,5,4,3,2,1
2,2,5,4,3,2,1
3,3,9,7,.,.,.
